# Segmenting and Clustering Neighborhoods in Toronto

## Question 1

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [5]:
import requests
!pip install BeautifulSoup4
from bs4 import BeautifulSoup
print('Imported')

     |████████████████████████████████| 122kB 5.5MB/s eta 0:00:01
Imported


In [6]:
URL=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(URL,'html.parser')
#print(soup.prettify())

In [7]:
PostalCodeList=[]
BoroughList=[]
NeighborhoodList=[]
for trs in soup.find('table').find_all('tr'):
    tds=trs.find_all('td')
    if len(tds)>0:
        PostalCodeList.append(tds[0].text.rstrip('\n'))
        BoroughList.append(tds[1].text.rstrip('\n'))
        NeighborhoodList.append(tds[2].text.rstrip('\n'))

In [8]:
toronto_df=pd.DataFrame({'PostalCode':PostalCodeList,
                       'Borough':BoroughList,
                       'Neighborhood':NeighborhoodList})
toronto_df.head()

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode           Borough               Neighborhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront

In [9]:
toronto_df.head(20)

AttributeError: 'NoneType' object has no attribute 'items'

   PostalCode           Borough  \
0         M1A      Not assigned   
1         M2A      Not assigned   
2         M3A        North York   
3         M4A        North York   
4         M5A  Downtown Toronto   
5         M6A        North York   
6         M7A  Downtown Toronto   
7         M8A      Not assigned   
8         M9A         Etobicoke   
9         M1B       Scarborough   
10        M2B      Not assigned   
11        M3B        North York   
12        M4B         East York   
13        M5B  Downtown Toronto   
14        M6B        North York   
15        M7B      Not assigned   
16        M8B      Not assigned   
17        M9B         Etobicoke   
18        M1C       Scarborough   
19        M2C      Not assigned   

                                         Neighborhood  
0                                        Not assigned  
1                                        Not assigned  
2                                           Parkwoods  
3                                    Vic

In [10]:
toronto_df1=toronto_df[toronto_df['Borough']!='Not assigned'].reset_index(drop=True)
toronto_df1.head()

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [11]:
toronto_df2 = toronto_df1.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df2.head()

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae

In [12]:
toronto_df2.loc[toronto_df2['Neighborhood']=='Not assigned','Neighborhood']=toronto_df2['Borough']
toronto_df2.head()

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae

In [13]:
toronto_df_t=toronto_df2
toronto_df_t.head()

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae

In [14]:
toronto_df_t.shape

(103, 3)

## Question 2

In [15]:
toronto_latlog=pd.read_csv('https://cocl.us/Geospatial_data')
toronto_latlog.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [16]:
toronto_latlog.rename(columns={'Postal Code':'PostalCode'},inplace=True)
toronto_latlog.head()

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

In [17]:
toronto_df_mer=toronto_df_t.merge(toronto_latlog,on='PostalCode',how='left')
toronto_df_mer.head()

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  

In [18]:
column_names=['PostalCode','Borough','Neighborhood','Latitude','Longitude']
P_list=['M5G','M2H','M4B','M1J','M4G','M4M','M1R','M9L','M5V','M1B','M5A']
df=pd.DataFrame(columns=column_names)
for p_code in P_list:
    df=df.append(toronto_df_mer[toronto_df_mer['PostalCode']==p_code],ignore_index=True)
df

AttributeError: 'NoneType' object has no attribute 'items'

   PostalCode           Borough  \
0         M5G  Downtown Toronto   
1         M2H        North York   
2         M4B         East York   
3         M1J       Scarborough   
4         M4G         East York   
5         M4M      East Toronto   
6         M1R       Scarborough   
7         M9L        North York   
8         M5V  Downtown Toronto   
9         M1B       Scarborough   
10        M5A  Downtown Toronto   

                                         Neighborhood   Latitude  Longitude  
0                                  Central Bay Street  43.657952 -79.387383  
1                                   Hillcrest Village  43.803762 -79.363452  
2                     Parkview Hill, Woodbine Gardens  43.706397 -79.309937  
3                                 Scarborough Village  43.744734 -79.239476  
4                                             Leaside  43.709060 -79.363452  
5                                     Studio District  43.659526 -79.340923  
6                                

## Question 3

In [19]:
address='Toronto'

geolocator=Nominatim(user_agent="tor_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geografical coordinates of Toronto are {},{}.'.format(latitude,longitude))

The geografical coordinates of Toronto are 43.6534817,-79.3839347.


In [20]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_mer['Latitude'], toronto_df_mer['Longitude'], toronto_df_mer['Borough'], toronto_df_mer['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [47]:
CLIENT_ID = 'Client-id' # your Foursquare ID
CLIENT_SECRET = 'Client-server' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Client-id
CLIENT_SECRET:Client-server


### Near by venues

In [22]:
def getNearbyVenues(df_tor, radius=500, LIMIT=100):
    
    venues_list=[]
    for postal, borough, neigh, lat, lng, in zip(df_tor['PostalCode'], df_tor['Borough'], df_tor["Neighborhood"], df_tor['Latitude'], df_tor['Longitude']):   
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postal,
            borough,
            neigh,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode',
                             'Borough',
                             'Neighborhood',
                             'Latitude',
                             'Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

In [25]:
venues_df=getNearbyVenues(toronto_df_mer)
venues_df.head()

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1B  Scarborough                          Malvern, Rouge  43.806686   
2        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
3        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
4        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   

   Longitude                   Venue  Venue Latitude  Venue Longitude  \
0 -79.194353                 Wendy’s       43.807448       -79.199056   
1 -79.194353   Interprovincial Group       43.805630       -79.200378   
2 -79.160497  Chris Effects Painting       43.784343       -79.163742   
3 -79.160497   Royal Canadian Legion       43.782533       -79.163085   
4 -79.188711          RBC Royal Bank       43.766790       -79.191151   

               Venue Category  
0        Fast Food Restaurant  
1               

In [26]:
venues_df.shape

(2153, 9)

In [27]:
venues_df.groupby(['Borough']).count()

AttributeError: 'NoneType' object has no attribute 'items'

                  PostalCode  Neighborhood  Latitude  Longitude  Venue  \
Borough                                                                  
Central Toronto          111           111       111        111    111   
Downtown Toronto        1252          1252      1252       1252   1252   
East Toronto             123           123       123        123    123   
East York                 74            74        74         74     74   
Etobicoke                 75            75        75         75     75   
Mississauga               13            13        13         13     13   
North York               241           241       241        241    241   
Scarborough               88            88        88         88     88   
West Toronto             158           158       158        158    158   
York                      18            18        18         18     18   

                  Venue Latitude  Venue Longitude  Venue Category  
Borough                                    

In [28]:
print('There are {} uniques categories'.format(len(venues_df['Venue Category'].unique())))

There are 269 uniques categories


### Analysising each Neighborhood

In [30]:
venues_onehot=pd.get_dummies(venues_df['Venue Category'],prefix='',prefix_sep='')
venues_onehot['Neighborhood']=venues_df['Neighborhood']
fixed_columns=[venues_onehot.columns[-1]]+list(venues_onehot.columns[:-1])
venues_onehot=venues_onehot[fixed_columns]
venues_onehot.head()

AttributeError: 'NoneType' object has no attribute 'items'

   Yoga Studio  Accessories Store  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Art Museum  Arts

In [31]:
venues_onehot.shape

(2153, 269)

In [32]:
venues_grouped=venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped.head()

AttributeError: 'NoneType' object has no attribute 'items'

                                      Neighborhood  Yoga Studio  \
0                                        Agincourt          0.0   
1                           Alderwood, Long Branch          0.0   
2  Bathurst Manor, Wilson Heights, Downsview North          0.0   
3                                  Bayview Village          0.0   
4                Bedford Park, Lawrence Manor East          0.0   

   Accessories Store  Airport  Airport Food Court  Airport Gate  \
0                0.0      0.0                 0.0           0.0   
1                0.0      0.0                 0.0           0.0   
2                0.0      0.0                 0.0           0.0   
3                0.0      0.0                 0.0           0.0   
4                0.0      0.0                 0.0           0.0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0             0.0              0.0               0.0             0.000000   
1             0.0              0.0      

In [33]:
venues_grouped.shape

(95, 269)

In [34]:
num_top_venues = 5

for hood in venues_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = venues_grouped[venues_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4              Metro Station  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.25
1  Sandwich Place  0.12
2    Skating Rink  0.12
3             Gym  0.12
4     Coffee Shop  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0    Coffee Shop  0.10
1           Bank  0.10
2    Pizza Place  0.05
3  Shopping Mall  0.05
4           Park  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4                Motel  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2      Sandwich Place  0.09
3          Hobby Shop  0.09
4            Ph

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

AttributeError: 'NoneType' object has no attribute 'items'

                                      Neighborhood 1st Most Common Venue  \
0                                        Agincourt          Skating Rink   
1                           Alderwood, Long Branch           Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North           Coffee Shop   
3                                  Bayview Village                  Café   
4                Bedford Park, Lawrence Manor East           Coffee Shop   

       2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0  Latin American Restaurant        Breakfast Spot                Lounge   
1                        Gym          Skating Rink        Sandwich Place   
2                       Bank                  Park   Fried Chicken Joint   
3                       Bank   Japanese Restaurant    Chinese Restaurant   
4                 Hobby Shop        Sandwich Place    Italian Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0         Women

### Cluster Neighborhood

In [37]:
# set number of clusters
kclusters = 5

venues_grouped_clustering = venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df_mer

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  Cluster Labels       1st Most Common Venue  \
0 -79.194353             0.0        Fast Food Restaurant   
1 -79.160497             3.0  Construction & Landscaping   
2 -79.188711             0.0                  Restaurant   
3 -79.216917             0.0                 Coffee Shop   
4 -79.239476             0.0            Hakka Restaurant   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0            Print Shop         Women's Store            Donut Shop   
1               

In [39]:
toronto_merged.head()

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  Cluster Labels       1st Most Common Venue  \
0 -79.194353             0.0        Fast Food Restaurant   
1 -79.160497             3.0  Construction & Landscaping   
2 -79.188711             0.0                  Restaurant   
3 -79.216917             0.0                 Coffee Shop   
4 -79.239476             0.0            Hakka Restaurant   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0            Print Shop         Women's Store            Donut Shop   
1               

In [40]:
toronto_merged=toronto_merged.dropna()
toronto_merged.head()

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  Cluster Labels       1st Most Common Venue  \
0 -79.194353             0.0        Fast Food Restaurant   
1 -79.160497             3.0  Construction & Landscaping   
2 -79.188711             0.0                  Restaurant   
3 -79.216917             0.0                 Coffee Shop   
4 -79.239476             0.0            Hakka Restaurant   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0            Print Shop         Women's Store            Donut Shop   
1               

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

AttributeError: 'NoneType' object has no attribute 'items'

              Borough  Cluster Labels   1st Most Common Venue  \
0         Scarborough             0.0    Fast Food Restaurant   
2         Scarborough             0.0              Restaurant   
3         Scarborough             0.0             Coffee Shop   
4         Scarborough             0.0        Hakka Restaurant   
6         Scarborough             0.0        Department Store   
8         Scarborough             0.0     American Restaurant   
9         Scarborough             0.0         College Stadium   
10        Scarborough             0.0       Indian Restaurant   
11        Scarborough             0.0              Smoke Shop   
12        Scarborough             0.0            Skating Rink   
13        Scarborough             0.0             Pizza Place   
15        Scarborough             0.0    Fast Food Restaurant   
17         North York             0.0                    Pool   
18         North York             0.0          Clothing Store   
19         North York    

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

AttributeError: 'NoneType' object has no attribute 'items'

             Borough  Cluster Labels 1st Most Common Venue  \
14       Scarborough             1.0                  Park   
21        North York             1.0                  Park   
23        North York             1.0     Convenience Store   
25        North York             1.0     Food & Drink Shop   
50  Downtown Toronto             1.0                  Park   
64   Central Toronto             1.0                  Park   
74              York             1.0                  Park   
98              York             1.0         Jewelry Store   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
14            Playground         Women's Store            Donut Shop   
21         Women's Store            Donut Shop    Dim Sum Restaurant   
23                  Park             Drugstore                 Diner   
25                  Park         Women's Store                 Diner   
50            Playground                 Trail     Electronics Store   
64       

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

AttributeError: 'NoneType' object has no attribute 'items'

            Borough  Cluster Labels 1st Most Common Venue  \
63  Central Toronto             2.0                Garden   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
63            Donut Shop    Dim Sum Restaurant                 Diner   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
63        Discount Store   Distribution Center               Dog Run   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
63      Doner Restaurant             Drugstore         Farmers Market  

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

AttributeError: 'NoneType' object has no attribute 'items'

       Borough  Cluster Labels       1st Most Common Venue  \
1  Scarborough             3.0  Construction & Landscaping   
5  Scarborough             3.0                  Playground   

        2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
1                         Bar         Women's Store   Dumpling Restaurant   
5  Construction & Landscaping         Women's Store            Donut Shop   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
1   Distribution Center               Dog Run      Doner Restaurant   
5                 Diner        Discount Store   Distribution Center   

  8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
1            Donut Shop             Drugstore  Eastern European Restaurant  
5               Dog Run      Doner Restaurant          Dumpling Restaurant  

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

AttributeError: 'NoneType' object has no attribute 'items'

             Borough  Cluster Labels 1st Most Common Venue  \
7        Scarborough             4.0                Bakery   
40         East York             4.0          Intersection   
44   Central Toronto             4.0                  Park   
48   Central Toronto             4.0                  Park   
79        North York             4.0                  Park   
90         Etobicoke             4.0                  Park   
100        Etobicoke             4.0                  Park   

    2nd Most Common Venue       3rd Most Common Venue 4th Most Common Venue  \
7          Ice Cream Shop                        Park          Soccer Field   
40                   Park           Convenience Store         Women's Store   
44            Swim School                    Bus Line      Doner Restaurant   
48                 Lawyer                Tennis Court            Restaurant   
79                 Bakery  Construction & Landscaping      Basketball Court   
90                   Pool    